<a href="https://colab.research.google.com/github/Sowmyad15/SMS_Spam/blob/main/BERT_MLP_Balanced%2BNewData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel


In [3]:
df=pd.read_csv('/content/spam.csv',encoding='latin-1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:

df = df.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
df.columns = ["label", "message"]
df['label_num']=df['label'].map({
    'ham':0,
    'spam':1
})
df.head()

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:

df_spam = df[df['label']=='spam']

df_ham = df[df['label']=='ham']

print("Ham Dataset Shape:", df_ham.shape)

print("Spam Dataset Shape:", df_spam.shape)

Ham Dataset Shape: (4825, 3)
Spam Dataset Shape: (747, 3)


# **Balanced Data- BERT+MLP**

In [6]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 3)

In [7]:

df= pd.concat([df_spam , df_ham_downsampled])

In [8]:
df['label_num'].value_counts()

1    747
0    747
Name: label_num, dtype: int64

In [9]:

import spacy

nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)
df['preprocessed_txt'] = df['message'].apply(preprocess)
df.head()

,label,message,label_num,preprocessed_txt
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry 2 wkly comp win FA Cup final tkts 2...
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,FreeMsg hey darle 3 week word like fun tb ok X...
8,spam,WINNER!! As a valued network customer you have...,1,WINNER value network customer select receivea ...
9,spam,Had your mobile 11 months or more? U R entitle...,1,mobile 11 month u r entitle update late colour...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1,"chance win cash 100 20,000 pound txt > CSH11 s..."


In [10]:

import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [11]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:
def get_bert_embeddings(texts):
    texts = texts.tolist() if isinstance(texts, pd.Series) else texts
    input_ids = []
    attention_masks = []

    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            max_length=128,
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt',
            pad_to_max_length=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    with torch.no_grad():
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        outputs = model(input_ids, attention_mask=attention_masks)
        pooled_output = outputs.pooler_output

    return pooled_output



In [13]:
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [14]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [15]:
bert_embeddings = get_bert_embeddings(df['preprocessed_txt'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
bert_embeddings[0]

tensor([-6.7605e-01, -4.1760e-01, -7.4975e-01,  4.9545e-01,  3.8311e-01,
        -1.5031e-01,  5.8715e-01,  3.0121e-01, -3.2861e-01, -9.9996e-01,
        -1.6286e-01,  5.3948e-01,  9.5133e-01,  2.2359e-01,  7.0963e-01,
        -3.5806e-01,  3.0650e-01, -4.8774e-01,  3.0125e-01,  2.8047e-01,
         5.6219e-01,  9.9985e-01,  2.0910e-01,  2.8976e-01,  3.9732e-01,
         6.8742e-01, -4.1467e-01,  7.6198e-01,  8.9191e-01,  5.6897e-01,
        -2.4566e-01,  2.9931e-01, -9.6557e-01, -2.4548e-01, -8.0565e-01,
        -9.8079e-01,  2.6141e-01, -5.1607e-01, -2.6123e-02, -3.2012e-02,
        -7.1511e-01,  3.9629e-01,  9.9987e-01, -1.1536e-01,  1.9850e-01,
        -2.3424e-01, -9.9998e-01,  1.3982e-01, -6.9615e-01,  5.3830e-01,
         3.2594e-01,  6.7014e-01,  2.0668e-01,  3.4313e-01,  3.3484e-01,
         2.9076e-01, -1.0411e-01,  3.5477e-02, -2.2749e-01, -5.9219e-01,
        -5.7355e-01,  5.1930e-01, -5.5766e-01, -7.8905e-01,  4.5395e-01,
         5.7246e-01, -2.4104e-01, -3.3214e-01, -2.9

In [17]:

X_train, X_test, y_train, y_test = train_test_split(
    bert_embeddings,
    df.label_num,
    test_size=0.2,
    random_state=42
)


In [18]:
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)

<ipython-input-18-663f12ca6201>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor=torch.tensor(X_train,dtype=torch.float32)


In [19]:
y_train_tensor=torch.tensor(y_train.to_numpy(),dtype=torch.float32)

In [20]:
y_test.value_counts()

0    154
1    145
Name: label_num, dtype: int64

In [21]:
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)


<ipython-input-21-b10cc042634a>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor=torch.tensor(X_test,dtype=torch.float32)


In [22]:
y_test_tensor=torch.tensor(y_test.to_numpy(),dtype=torch.float32)

In [23]:
print(y_test_tensor)

tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
        0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
        0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0.,
        0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
        0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
        0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 0., 0., 1., 1., 

In [24]:
input_size = 768
hidden_size = 512
output_size = 1

mlp_model = MLP(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)

In [25]:
criterion.to(device)

BCELoss()

In [26]:
mlp_model.to(device)

MLP(
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [27]:
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

In [28]:
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [29]:
epochs = 100
for epoch in range(epochs):
    mlp_model.train()
    optimizer.zero_grad()
    outputs = mlp_model(X_train_tensor).to(device)
    loss = criterion(outputs.squeeze(), y_train_tensor).to(device)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 0.7172061204910278
Epoch 2/100, Loss: 0.8024324178695679
Epoch 3/100, Loss: 0.615325927734375
Epoch 4/100, Loss: 0.6232313513755798
Epoch 5/100, Loss: 0.6319529414176941
Epoch 6/100, Loss: 0.5763626098632812
Epoch 7/100, Loss: 0.5472009778022766
Epoch 8/100, Loss: 0.5560793280601501
Epoch 9/100, Loss: 0.563433051109314
Epoch 10/100, Loss: 0.5507257580757141
Epoch 11/100, Loss: 0.5281355977058411
Epoch 12/100, Loss: 0.513615608215332
Epoch 13/100, Loss: 0.5125135779380798
Epoch 14/100, Loss: 0.5133048295974731
Epoch 15/100, Loss: 0.5044668912887573
Epoch 16/100, Loss: 0.48836052417755127
Epoch 17/100, Loss: 0.47478267550468445
Epoch 18/100, Loss: 0.46866023540496826
Epoch 19/100, Loss: 0.4656737446784973
Epoch 20/100, Loss: 0.45860326290130615
Epoch 21/100, Loss: 0.4461558759212494
Epoch 22/100, Loss: 0.4337446391582489
Epoch 23/100, Loss: 0.42624107003211975
Epoch 24/100, Loss: 0.42158085107803345
Epoch 25/100, Loss: 0.4136336147785187
Epoch 26/100, Loss: 0.401455402

In [30]:
 y_test_numpy = y_test_tensor.cpu().numpy()

In [31]:
mlp_model.eval()
with torch.no_grad():
    test_outputs = mlp_model(X_test_tensor)
    predictions = (test_outputs.squeeze() >= 0.5).cpu().float()
    accuracy = accuracy_score(y_test_numpy, predictions)
    print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 96.32%


# Trying with new data

In [33]:
sample=pd.read_csv("/content/new.csv.csv",encoding="latin-1")

In [34]:
sample.head()

,Message,Label
0,"Dear Student, Its never too late-clear your ba...",spam
1,Join V-STUDY and score excellent marks in clas...,spam
2,"Join crash courses for B.ST,A/C'S,ECO,ENG,&IP ...",spam
3,CRASH COURSES by BEST POOL OF FACULTY. ENGLISH...,spam
4,"Dear Ola Shuttle user, get 60% Off on your nex...",ham


In [37]:
s_bert_embeddings = get_bert_embeddings(sample['Message'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [38]:

sample['label_num']=sample['Label'].map({
    'ham':0,
    'spam':1
})
sample.head()

,Message,Label,label_num
0,"Dear Student, Its never too late-clear your ba...",spam,1
1,Join V-STUDY and score excellent marks in clas...,spam,1
2,"Join crash courses for B.ST,A/C'S,ECO,ENG,&IP ...",spam,1
3,CRASH COURSES by BEST POOL OF FACULTY. ENGLISH...,spam,1
4,"Dear Ola Shuttle user, get 60% Off on your nex...",ham,0


In [39]:
s_bert_embeddings[0]

tensor([-0.7404, -0.4187, -0.7663,  0.5865,  0.5280, -0.0290,  0.2990,  0.4203,
        -0.2872, -0.9999, -0.0392,  0.6326,  0.9817,  0.3596,  0.6809, -0.1318,
         0.3285, -0.6206,  0.2993,  0.6728,  0.6307,  0.9999,  0.3470,  0.3465,
         0.5044,  0.8668, -0.4992,  0.8461,  0.9159,  0.5942, -0.3155,  0.3458,
        -0.9894, -0.2778, -0.8039, -0.9882,  0.3752, -0.5275, -0.0979,  0.0451,
        -0.8430,  0.4018,  0.9999, -0.4998,  0.4638, -0.2741, -1.0000,  0.2254,
        -0.7735,  0.4480,  0.4250,  0.3735,  0.1315,  0.3174,  0.3065,  0.3676,
        -0.1133,  0.1731, -0.3213, -0.4847, -0.6546,  0.5370, -0.6659, -0.8175,
         0.2632,  0.5116, -0.0981, -0.3718, -0.0628, -0.0066,  0.4014,  0.3752,
         0.0492, -0.8989, -0.1756,  0.3947, -0.4519,  1.0000, -0.0727, -0.9612,
         0.8378,  0.2646,  0.3632,  0.3499,  0.1182, -1.0000,  0.2221, -0.1184,
        -0.9873,  0.2807,  0.4879, -0.2152,  0.7075,  0.3638, -0.3902, -0.4004,
        -0.0890, -0.5248, -0.5509, -0.31

In [40]:
X=torch.tensor(s_bert_embeddings,dtype=torch.float32)

<ipython-input-40-d2a0d459a899>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X=torch.tensor(s_bert_embeddings,dtype=torch.float32)


In [41]:
Y=torch.tensor(sample['label_num'],dtype=torch.float32)

In [43]:
mlp_model.eval()
with torch.no_grad():
    test_outputs = mlp_model(X)
    predictions = (test_outputs.squeeze() >= 0.5).cpu().float()
    accuracy = accuracy_score(Y, predictions)
    print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 83.70%
